In [148]:
import os
import pandas as pd
from multiprocessing import Pool
import tqdm
import re

#Importando programação criada pelo Thiago.
import files_indexing

pd.set_option('display.max_colwidth', -1)

In [2]:
def acessar_arquivos(Tema = 'All'):
    
    """Lista todos os caminhos possíveis da pasta raiz até todas as subpastas, acessa todos esses caminhos 
       e recupera as informações que estão em ppt ou word.
       
    param:
     Tema: Caso precise ser pego só um tema de trabalho, fazer com que Tema seja igual ao nome da pasta do tema relacionado
    
    """
    
    # Objetos que iram receber as informações finais 
    dados_finais = {}
    arquivos_error = {}
    
    if Tema == "All":
        
        #O for a seguir será um conunto de listas, cada lista possui todos os caminhos de um determinado tema
        lista_path = [[x[0] for x in os.walk(i)] for i in os.listdir() if ('.' not in i[0] and '.ipynb' not in i and '\uf00d' not in i)]
        
        #Primeiro for é para passar na lista de temas e o segundo em todos caminhos de cada tema
        for tema in tqdm.tqdm(lista_path, total = len(lista_path)):
            for path in tqdm.tqdm(tema, total = len(tema)):
                
                dados_finais.update(arquivos_da_pasta(path)[0])
                arquivos_error.update(arquivos_da_pasta(path)[1])
        
    else:
        
        #Possui um for a menos, porque já começa do Tema
        lista_path = [x[0] for x in os.walk(Tema)]
        
        for path in tqdm.tqdm(lista_path, total = len(lista_path)):
                dados_finais.update(arquivos_da_pasta(path)[0])
                arquivos_error.update(arquivos_da_pasta(path)[1])
                                            
    return dados_finais, arquivos_error

In [4]:
def arquivos_da_pasta(path):
    
    """Dado um caminho ele recupera todas as informações no formato de ppt daquele caminho
    
    param:
     path: Caminho de onde estão os arquivos que você quer recuperar
    """
    
    #Objetos para receberem valores
    dados_path = {}
    error = {}
    
    #Pegando o nome de todos os arquivos, vendo quais são maiores que 10kb e os que tem extensão doc, docx, ppt, pptx
    aux_arquivos = os.listdir(path)
    aux_arquivos = [arquivo for arquivo in aux_arquivos if os.stat(os.path.join(path,arquivo)).st_size > 10240]
    aux_arquivos = [arquivo for arquivo in aux_arquivos if '.ppt' in arquivo or '.doc' in arquivo]
    
    
    for documento in aux_arquivos:
        
        if '.ppt' in documento:
            
            #Irá testar a função criado pelo Thiago, caso der erro essa função salva o nome do arquivo e o caminho
            try: 
                texto = files_indexing.get_data_pptx(os.path.join(path, documento))
                dados_path.update({documento: texto})
            except:
                error.update({documento: path})
                
        else:
            
            #Irá testar a função criado pelo Thiago, caso der erro essa função salva o nome do arquivo e o caminho
            try: 
                texto = files_indexing.get_data_docx(os.path.join(path, documento))
                dados_path.update({documento: texto})
            except:
                error.update({documento: path})
            
            
    return dados_path, error

In [ ]:
teste_dados, teste_error = acessar_arquivos('Shopping Center')

  2%|▏         | 22/950 [00:15<54:29,  3.52s/it]

In [13]:
len(teste_dados.keys())

527

In [11]:
len(teste_corrompidos.keys())

45

In [12]:
teste = ['aaa.pptx', 'sdasd', 'teste.doc', 'vai']

In [15]:
aux_arquivos = [arquivo for arquivo in teste if '.ppt' in arquivo or '.doc' in arquivo]

Primeiro, vamos identificar quais documentos dentro da coleção total devem conter o tipo de informação que estamos procurando para indexar. Para isso, vamos identificar todos os documentos do tipo doc, docx, ppt, pptx e pdf.

In [112]:
"""Lista todos os caminhos possíveis da pasta raiz até todas as subpastas, acessa todos esses caminhos 
   e recupera as informações que estão em ppt ou word.

param:
 Tema: Caso precise ser pego só um tema de trabalho, fazer com que Tema seja igual ao nome da pasta do tema relacionado

"""

searched_types = ['.doc', '.docx', '.ppt', '.pptx', '.pdf']

#cria o dataframe
df = pd.DataFrame(columns=('file','type','size'))


#O for a seguir será um conunto de listas, cada lista possui todos os caminhos de um determinado tema
lista_path = [[x[0] for x in os.walk(i)] for i in os.listdir() if ('.' not in i[0] and '.ipynb' not in i and '\uf00d' not in i)]

#Primeiro for é para passar na lista de temas e o segundo em todos caminhos de cada tema
for tema in tqdm.tqdm(lista_path, total = len(lista_path)):
    for path in tqdm.tqdm(tema, total = len(tema)):
        
        #Pegando o nome de todos os arquivos, vendo quais são maiores que 10kb e os que tem extensão doc, docx, ppt, pptx
        aux_arquivos = os.listdir(path)
        aux_arquivos = [arquivo for arquivo in aux_arquivos if os.stat(os.path.join(path,arquivo)).st_size > 10240]
        aux_arquivos = [arquivo for arquivo in aux_arquivos if any(type in arquivo for type in searched_types)]

        for documento in aux_arquivos:

            df = df.append({'file': os.path.join(path, documento),
                            'type': documento[(documento.rfind('.') + 1):],
                            'size': os.stat(os.path.join(path, documento)).st_size/1000000},
                           ignore_index = True
                          )

            
#retira documentos em formatos indesejado que passaram no filtro
df = df[df['type'] != 'xlsx']
df = df[df['type'] != 'ppsx']

 27%|██▋       | 567/2086 [00:02<00:08, 184.74it/s]


 70%|███████   | 7/10 [00:48<00:18,  6.24s/it]t/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]

100%|██████████| 10/10 [00:48<00:00,  4.82s/it]


In [169]:
#Adicionando ao dataframe o codigo do projeto

def get_project(file):
    pos_start, pos_end = [m.start() for m in re.finditer(r"/",file)][1:3]
    return file[(pos_start+1):pos_end]

df['project'] = df.file.apply(lambda x: get_project(x))

Algumas eststísticas dos documentos filtrados

In [184]:
print('Tamanho total dos documentos: {0:.2f} gb'.format(sum(df['size']/1000)))
print('')

print('Qtd de projetos: {}'.format(len(df.project.unique())))
print('')

print('Distribuição do número dedocumentos por tipo:')
df.groupby('type').count()

Tamanho total dos documentos: 30.47 gb

Qtd de projetos: 110

Distribuição do número dedocumentos por tipo:


,file,size,project
type,,,
doc,1364,1364,1364
docx,1353,1353,1353
pdf,4574,4574,4574
ppt,905,905,905
pptm,23,23,23
pptx,5313,5313,5313


In [185]:
df.project.unique()

array(['5739', '0557-12', '0626-11', '1866',
       '2012_Engeform_GPD e Diagn_GC', '2013_ 1506-13 Engeform GREI_ GC',
       '3082', '3133', '4511', '4932', '0450-12', '0637_11', '2158',
       '0645-12 Argos Colombia', '0645-12 Grupo Argos', '1967', '2625',
       '3454', '4332', '1153', '2662', 'Projeto de Gestão VC - 2011',
       '3281', '0095-12', '0811-09', '2607', '3708', '0672-12 Caparaó',
       '1312 Caparaó', '2538', '4911', '1060', '1905', '0277-12 COSIL',
       '0674-10', '5060', '9887', '0315-12', '0242-11', '0568-12', '1970',
       '3237', '3247', '0581-10', '1583', '1861', '2040', '2049', '2793',
       '0030-12', '2041', '3408', '3423', '4271', '0386-11', '3113',
       '4555', '0119-11 Rogga S_A', '1215', '1888', '2144_3650', '3151',
       '3431', '3624', '3857', '0552-12 Terral', '1776 Terral',
       '0399-12 VIVER INCORPORADORA', '3524', '1308', '2536', '3034',
       '3965', '4844', '0685-11 Construtora Barbosa Mello',
       '1043 - Queiroz Galvão', '0828

In [166]:
get_project(texto)

'5739'